In [1]:
import sys
sys.path.append("/Users/hirototakaura/opt/anaconda3/lib/python3.8/site-packages")
from osgeo import gdal, ogr
from geopy.distance import geodesic
from PIL import Image
import pandas as pd
import numpy as np
import argparse
import re
import glob
import MakeBoxLayer

検出したbboxの座標変換と
重なってるboxの除去、小さいboxの除去を行う

In [11]:
'''
LINESTRINGからPLOYGONに変換する関数
line : 変換対象のオブジェクト
df: 変換対象のオブジェクトを含むdataframe
crt : 変換対象のオブジェクトを含む行番号
wrtcol: 書き込む列名
'''
def makePolygon(line, df, crt, wrtcol):
    ring = ogr.Geometry(ogr.wkbLinearRing)
    list_x = []
    list_y = []
    #lineの座標をリストに
    for count in range(line.GetPointCount()): 
        cood = line.GetPoint(count)
        list_x.append(cood[0])
        list_y.append(cood[1])
    #stringlineに点を追加
    for x, y in zip(list_x, list_y):
        ring.AddPoint(x, y)
    
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    #line1を着目しているラインに書き換える
    df.at[crt, wrtcol] = poly
    #print("make polygon in df{}".format(crt))
        

def remDoubledBox(box_cs):
    box_df = pd.read_csv(box_cs)
    box_df = box_df.loc[:, ['WKT', 'x1y1', 'x2y2', 'x3y3', 'x4y4','classname', 'score']]
    #ポリゴン化した列を追加
    box_df['WKT'] = box_df['WKT'].map(lambda x: ogr.CreateGeometryFromWkt(x))
    box_df['Polygon'] = box_df['WKT']
    for col in box_df.index.to_numpy():
        makePolygon(box_df['WKT'][col], box_df, col, 'Polygon')
    #重複したboxを削除
    box_work = box_df
    threshold = 0.2
    count = 0
    rem = []
    for label_num in box_work.index.to_numpy():
        count += 1
        #label_numが既に削除されている行番号ならスキップ
        if label_num in rem:
            #print("label_num:{}".format(label_num)) 
            continue
        high_iou = []
        label = box_work['Polygon'][label_num]
        for box_num in box_work.index.to_numpy():
            #box_numが既に削除されている行番号ならスキップ
            if box_num in rem:
                print("skip1: {}".format(box_num))
                continue
            #labelとboxが同じならスキップ
            if box_num == label_num:
                continue
            box =  box_work['Polygon'][box_num]
            intersection = box.Intersection(label)
            if not intersection.IsEmpty():
                s1 = box.Union(label).GetArea()
                s2 = intersection.GetArea() 
                iou = s2 / s1
                #閾値以上はhigh_iouに追加
                if iou >= threshold:
                    high_iou.append(box_num)

        #閾値以上のiouが存在しないなら終了
        if len(high_iou) == 0:
            continue  
        high_iou.append(label_num) 
        min_idx = box_work.loc[high_iou, 'score'].idxmin()
        rem.append(min_idx)
        box_work.drop(min_idx, inplace=True)
        if count%10 == 0:
            print('-----now {0} processed -----'.format(count))
    print('{} boxes have been dropped \n done!'.format(len(rem)))
    
    return box_work
    
    
def strToList(xy):
    if '[' in xy:
        xy = xy[1:-1]
    xypair = xy.split(",")
    xypair = [float(i) for i in xypair]
    return xypair


#座標からLINESTRINGを作成。LINESTRINGオブジェクトを返す
def PointToLine(xypair):
    xlist = []
    ylist = []
    #各点をリストに
    for pair in xypair:
        xlist.append(pair[0])
        ylist.append(pair[1])
        
    xlist.append(xypair[0][0])
    ylist.append(xypair[0][1])
    #ラインを作成
    line = ogr.Geometry(ogr.wkbLineString)
    for x, y in zip(xlist, ylist):
        if (isinstance(x, float)) and (isinstance(y, float)):
            pass
        else:
            x = float(x)
            y = float(y)
            
        line.AddPoint(x, y)
        
    return line


def cal_distance(pair1, pair2):
    dis = geodesic(pair1, pair2).m
    return dis


def remSmallBox(box_rem, outputfule):
    #既に重複を取り除いて緯度経度化したbboxのファイルを読み込む
    dsCS = box_rem.copy()
    dsCS = dsCS.reset_index(drop=True)
    if isinstance(dsCS['x1y1'][0], str):
        for cood in ["x1y1", "x2y2", "x3y3", "x4y4"]:
            dsCS[cood] = dsCS[cood].map(lambda x : strToList(x))

    L1 = []
    L2 = []
    for col in dsCS.index.to_numpy():
        pair1 = dsCS['x1y1'][col]
        pair2 = dsCS['x2y2'][col]
        pair3 = dsCS['x3y3'][col]

        #(Longtitude, Latitude) -> (Latitude, Longtitude)
        pair1 = [pair1[1], pair1[0]]
        pair2 = [pair2[1], pair2[0]]
        pair3 = [pair3[1], pair3[0]]

        L1.append(cal_distance(pair1, pair2))
        L2.append(cal_distance(pair2, pair3)) 
    dsCS['L1'] = L1
    dsCS['L2'] = L2

    full_length = []
    wide_length = []
    for col in dsCS.index.to_numpy():
        l1 = dsCS['L1'][col]
        l2 = dsCS['L2'][col]
        if l1 >= l2:
            full_length.append(l1)
            wide_length.append(l2)
        else:
            full_length.append(l2)
            wide_length.append(l1)
    dsCS['FullLength'] = full_length
    dsCS['WideLength'] = wide_length

    small_remd = dsCS[(dsCS['FullLength'] > 3.0) & (dsCS['WideLength'] > 1.4)]

    small_remd = small_remd.reset_index(drop=True)    
    small_remd.to_csv(outputfile)

def main(img_path, txt_path, outputfile, convert = True, remSmall = True, remDouble = True, check = False, num_of_checks = 1):
    if convert == True:
        #まとめ用dic
        dic = {}

        files = glob.glob(txt_path)
        for file in files:
            filename = re.split('[/.]', file)[-2]
            imgpath = img_path + filename + '.jpg'
            tifpath = img_path+filename +'.tif'
            print(tifpath)
            cood_box = MakeBoxLayer.MakeBoxLayer(file, imgpath, tifpath)
            for id in cood_box.index.to_numpy():
                for key in cood_box.columns.values:
                    if not key in dic:
                        dic[key] = []
                    dic[key].append(cood_box[key][id])
    #緯度経度座標に変換したboxの保存場所
    name_list = re.split('[/]', outputfile)
    box_cs = '../RodeOutline_HigasiOsaka/bbox/bbox-csv/' + name_list[-3] + '/' + name_list[-2] + '/' + name_list[-2] + '_cs.csv'
    if convert == True:
        #dicをDataFrameに変換してbox_csに保存
        pd.DataFrame(dic).to_csv(box_cs)
        
    if remSmall == True:#remSmallをTrueにすると小さいbboxの除去を行う
        box_drem = remDoubledBox(box_cs)
        
    if remSmall == True:#remDoubleをTrueにすると重複したbboxの除去を行う
        box_sdrem = remSmallBox(box_drem, outputfile)
        
    #一回の除去で、すべてが小さいbboxと重複したbboxが、取り除かれない場合がある。なぜ一回ですべて取り除かれないかは不明
    if check == True:#checkをTrueにするとoutputに書き出したファイルに対してもういちど除去を行う
        print('-----check-----')
        print(f'\nnumber of checks is -----> {num_of_checks}')
        for num in range(num_of_checks):
            box_drem = remDoubledBox(outputfile)
            box_sdrem = remSmallBox(box_drem, outputfile)
    
if __name__ == "__main__":
    key = '1249'
    day = '20200528'
    #検出に使用したjpg、tiffの画像があるフォルダ
    img_path = f'../RodeOutline_HigasiOsaka/GoogleMap/rd{key}/glmap{day}/'
    #検出したbboxの座標がまとめてあるフォルダ
    txt_path = f'../RodeOutline_HigasiOsaka/bbox/bbox-txt/rd{key}/rd{key}_{day}_txt/*'
    #書き出しパス
    outputfile = f'../RodeOutline_HigasiOsaka/bbox/bbox-csv/rd{key}/rd{key}_{day}/rd{key}_{day}-SDrmvtest.csv'
    main(img_path, txt_path, outputfile, check=True)


../RodeOutline_HigasiOsaka/GoogleMap/rd1249/glmap20200528/rd1249_20200528_4.tif
../RodeOutline_HigasiOsaka/GoogleMap/rd1249/glmap20200528/rd1249_20200528_5.tif
../RodeOutline_HigasiOsaka/GoogleMap/rd1249/glmap20200528/rd1249_20200528_7.tif
../RodeOutline_HigasiOsaka/GoogleMap/rd1249/glmap20200528/rd1249_20200528_6.tif
../RodeOutline_HigasiOsaka/GoogleMap/rd1249/glmap20200528/rd1249_20200528_2.tif
../RodeOutline_HigasiOsaka/GoogleMap/rd1249/glmap20200528/rd1249_20200528_3.tif
../RodeOutline_HigasiOsaka/GoogleMap/rd1249/glmap20200528/rd1249_20200528_1.tif
../RodeOutline_HigasiOsaka/GoogleMap/rd1249/glmap20200528/rd1249_20200528_8.tif
-----now 10 processed -----
-----now 30 processed -----
-----now 70 processed -----
-----now 80 processed -----
-----now 90 processed -----
-----now 100 processed -----
-----now 110 processed -----
-----now 170 processed -----
-----now 220 processed -----
-----now 250 processed -----
-----now 260 processed -----
-----now 270 processed -----


Warning 1: OGR_G_Area() called against non-surface geometry type.
Warning 1: OGR_G_Area() called against non-surface geometry type.


-----now 340 processed -----
-----now 360 processed -----
-----now 370 processed -----
-----now 380 processed -----
-----now 390 processed -----
-----now 410 processed -----
-----now 420 processed -----
-----now 440 processed -----
-----now 540 processed -----
-----now 550 processed -----
175 boxes have been dropped 
 done!
-----check-----

number of checks is -----> 1
2 boxes have been dropped 
 done!
